# **1. Configuración del Ambiente**


---

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
global df_traffic, resultados, modelo, modelo_clasificacion

# **2. Creación de Modelo de Gradient Boosted Regressor**


---

In [2]:
df_traffic = pd.read_csv('Classifier.csv', sep=';')
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   visitNumber         12283 non-null  int64  
 1   browser             12283 non-null  int64  
 2   operatingSystem     12283 non-null  int64  
 3   deviceCategory      12283 non-null  int64  
 4   continent           12283 non-null  int64  
 5   country             12283 non-null  int64  
 6   metro               12283 non-null  int64  
 7   city                12283 non-null  int64  
 8   networkDomain       12283 non-null  int64  
 9   campaign            12283 non-null  int64  
 10  source              12283 non-null  int64  
 11  medium              12283 non-null  int64  
 12  keyword             12283 non-null  int64  
 13  referralPath        12283 non-null  int64  
 14  adContent           12283 non-null  int64  
 15  pageviews           12283 non-null  int64  
 16  boun

In [3]:
# Dividir los datos en conjunto de entrenamiento y prueba
X = df_traffic.drop(columns=['transactionRevenue'])
y = df_traffic['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

random_forest = RandomForestRegressor()
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 229.69109344336587


In [4]:
#definiendo el K - número de subconjuntos
cv = KFold(n_splits = 5, shuffle=True)

In [5]:
accuracy_cross_val = cross_val_score(estimator = random_forest,
                                       X = X_train,
                                       y = y_train,
                                       scoring = 'neg_mean_squared_error',
                                       cv = cv)
print('Accuracy promedio de mse: {}'.format(round(accuracy_cross_val.mean(), 3)))

Accuracy promedio de mse: -394.288


In [6]:
# Selección de hiperparámetros para la regresión lineal
param_random_forest = {
    'n_estimators': [50, 100, 200],
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],  # Si se deben utilizar muestras de arranque
    'oob_score': [True, False]   # Si se debe usar out-of-bag estimación
}

In [7]:
# Configurar RandomizedSearchCV
randomized_search_rfm = RandomizedSearchCV(
    estimator=random_forest,
    param_distributions=param_random_forest,
    n_iter=10,  
    cv=5, 
    scoring='neg_mean_squared_error',  
    n_jobs=-1, 
    random_state=42  
)


In [8]:
# Ajustar el modelo
randomized_search_rfm.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['squared_error',
                                                      'friedman_mse',
                                                      'absolute_error'],
                                        'max_depth': [None, 10, 20],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200],
                                        'oob_score': [True, False]},
                   random_state=42, scoring='neg_mean_squared_error')

In [9]:
# Obtener los mejores parámetros y la mejor puntuación
best_params_rf = randomized_search_rfm.best_params_
best_score_rf = randomized_search_rfm.best_score_

print("Mejores parámetros:", best_params_rf)
print("Mejor puntuación:", best_score_rf)

Mejores parámetros: {'oob_score': True, 'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20, 'criterion': 'squared_error', 'bootstrap': True}
Mejor puntuación: -313.65812741943466


In [10]:
# Crear una nueva instancia de GradientBoostingRegressor con los mejores parámetros encontrados
best_rf_model = RandomForestRegressor(**randomized_search_rfm.best_params_)

# Entrenar el modelo con los datos de entrenamiento
best_rf_model.fit(X_train, y_train)

# Predecir con el modelo entrenado
y_pred = best_rf_model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calcular R^2
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calcular RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 253.848535666906
R^2 Score: 0.408825682587321
Root Mean Squared Error: 15.932624883141697
